In [1]:
from sklearn.ensemble import GradientBoostingClassifier
import pickle
from sklearn.metrics import ndcg_score
import pandas as pd
import gc

In [2]:
# training set
x = pd.read_csv("data/dummy/train.csv")

# val set 20%
x_val = x.sample(n=int(0.1*len(x)))
x = x.drop(x_val.index)

## TODO: date_time
x.drop(['click_bool', 'booking_bool', 'date_time'], axis=1, inplace=True)
x_val.drop(['click_bool', 'booking_bool', 'date_time'], axis=1, inplace=True)

# sort both on search id
x.sort_values('srch_id', ignore_index=True)
x_val.sort_values('srch_id', ignore_index=True)

,srch_id,site_id,visitor_location_country_id,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,...,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,orig_destination_distance,random_bool,agg_comp_rate,agg_comp_perc,agg_comp_inv,target
0,1,12,187,219,88127,3,3.0,1,1.39,0.0038,...,4,0,1,1,-1.00,1,0.000000,0.0,0.0,0
1,4,5,219,219,58696,0,4.5,0,1.61,0.0823,...,1,0,1,1,236.91,1,0.000000,0.0,0.0,0
2,4,5,219,219,89119,4,4.5,1,3.18,0.2742,...,1,0,1,1,238.06,1,0.000000,0.0,0.0,0
3,4,5,219,219,11622,4,4.0,0,2.71,0.0000,...,1,0,1,1,238.34,1,0.000000,0.0,0.0,0
4,8,5,219,219,10250,3,3.5,1,1.39,0.0321,...,1,0,1,0,5.40,0,0.000000,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495829,332784,24,219,100,140304,3,4.0,0,4.55,0.1606,...,1,0,1,0,328.93,0,0.000000,0.0,0.0,0
495830,332784,24,219,100,78171,2,0.0,0,5.03,0.0000,...,1,0,1,0,329.20,0,0.000000,0.0,0.0,0
495831,332784,24,219,100,128677,3,3.5,1,4.56,0.1475,...,1,0,1,0,328.72,0,0.333333,18.0,0.0,0
495832,332784,24,219,100,50151,5,4.5,0,4.85,0.0990,...,1,0,1,0,329.06,0,0.000000,0.0,0.0,0


In [3]:
y_val = x_val.target.squeeze()
y = x.target.squeeze()
x.drop(['target'], axis=1, inplace=True)
x_val.drop(['target'], axis=1, inplace=True)

In [4]:
groups = x.srch_id.value_counts().sort_index()
groups_val = x_val.srch_id.value_counts().sort_index()

In [5]:
import lightgbm

# default lightgbm model with sklearn api
gbm = lightgbm.LGBMRanker(metric="ndcg", objective="lambdarank", num_boost_round=1000) 

#label_gain

##rank_xendcg, XE_NDCG_MART ranking objective function, aliases: xendcg, xe_ndcg, xe_ndcg_mart, xendcg_mart
#rank_xendcg is faster than and achieves the similar performance as lambdarank

# fitting model 
gbm.fit(
    x,
    y,
    eval_set = [(x_val, y_val)],
    group = groups,
    eval_group = [groups_val],
    verbose=1,
    early_stopping_rounds=100
)

/home/susy/anaconda3/lib/python3.8/site-packages/lightgbm/engine.py:151: UserWarning: Found `num_boost_round` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[LightGBM] [Warning] num_iterations is set=1000, num_boost_round=1000 will be ignored. Current value: num_iterations=1000
[1]	valid_0's ndcg@1: 0.939736	valid_0's ndcg@2: 0.957723	valid_0's ndcg@3: 0.966154	valid_0's ndcg@4: 0.969983	valid_0's ndcg@5: 0.971524
Training until validation scores don't improve for 100 rounds
[2]	valid_0's ndcg@1: 0.942441	valid_0's ndcg@2: 0.96022	valid_0's ndcg@3: 0.968347	valid_0's ndcg@4: 0.9718	valid_0's ndcg@5: 0.973082
[3]	valid_0's ndcg@1: 0.943699	valid_0's ndcg@2: 0.960986	valid_0's ndcg@3: 0.969	valid_0's ndcg@4: 0.972436	valid_0's ndcg@5: 0.973667
[4]	valid_0's ndcg@1: 0.944244	valid_0's ndcg@2: 0.961398	valid_0's ndcg@3: 0.969228	valid_0's ndcg@4: 0.97264	valid_0's ndcg@5: 0.973926
[5]	valid_0's ndcg@1: 0.94458	valid_0's ndcg@2: 0.961579	valid_0's ndcg@3: 0.969345	valid_0's ndcg@4: 0.972764	valid_0's ndcg@5: 0.974058
[6]	valid_0's ndcg@1: 0.944815	valid_0's ndcg@2: 0.961797	valid_0's ndcg@3: 0.969499	valid_0's ndcg@4: 0.972901	valid_0's ndcg@5:

[60]	valid_0's ndcg@1: 0.945095	valid_0's ndcg@2: 0.962324	valid_0's ndcg@3: 0.969752	valid_0's ndcg@4: 0.973117	valid_0's ndcg@5: 0.974372
[61]	valid_0's ndcg@1: 0.94514	valid_0's ndcg@2: 0.962367	valid_0's ndcg@3: 0.969755	valid_0's ndcg@4: 0.973145	valid_0's ndcg@5: 0.97439
[62]	valid_0's ndcg@1: 0.945164	valid_0's ndcg@2: 0.962412	valid_0's ndcg@3: 0.969742	valid_0's ndcg@4: 0.973163	valid_0's ndcg@5: 0.974411
[63]	valid_0's ndcg@1: 0.945072	valid_0's ndcg@2: 0.962331	valid_0's ndcg@3: 0.969723	valid_0's ndcg@4: 0.97312	valid_0's ndcg@5: 0.974365
[64]	valid_0's ndcg@1: 0.945066	valid_0's ndcg@2: 0.962326	valid_0's ndcg@3: 0.96971	valid_0's ndcg@4: 0.97313	valid_0's ndcg@5: 0.974367
[65]	valid_0's ndcg@1: 0.944998	valid_0's ndcg@2: 0.962329	valid_0's ndcg@3: 0.969689	valid_0's ndcg@4: 0.973106	valid_0's ndcg@5: 0.974341
[66]	valid_0's ndcg@1: 0.944969	valid_0's ndcg@2: 0.962278	valid_0's ndcg@3: 0.969668	valid_0's ndcg@4: 0.973077	valid_0's ndcg@5: 0.974323
[67]	valid_0's ndcg@1: 0.

[120]	valid_0's ndcg@1: 0.944656	valid_0's ndcg@2: 0.962004	valid_0's ndcg@3: 0.969511	valid_0's ndcg@4: 0.972919	valid_0's ndcg@5: 0.974167
[121]	valid_0's ndcg@1: 0.94465	valid_0's ndcg@2: 0.961978	valid_0's ndcg@3: 0.969472	valid_0's ndcg@4: 0.972908	valid_0's ndcg@5: 0.974159
[122]	valid_0's ndcg@1: 0.944696	valid_0's ndcg@2: 0.961986	valid_0's ndcg@3: 0.969497	valid_0's ndcg@4: 0.972915	valid_0's ndcg@5: 0.974178
[123]	valid_0's ndcg@1: 0.944645	valid_0's ndcg@2: 0.962002	valid_0's ndcg@3: 0.969472	valid_0's ndcg@4: 0.972903	valid_0's ndcg@5: 0.974161
[124]	valid_0's ndcg@1: 0.944628	valid_0's ndcg@2: 0.961983	valid_0's ndcg@3: 0.969479	valid_0's ndcg@4: 0.972896	valid_0's ndcg@5: 0.974157
[125]	valid_0's ndcg@1: 0.94461	valid_0's ndcg@2: 0.961976	valid_0's ndcg@3: 0.969475	valid_0's ndcg@4: 0.972891	valid_0's ndcg@5: 0.974155
[126]	valid_0's ndcg@1: 0.944565	valid_0's ndcg@2: 0.961935	valid_0's ndcg@3: 0.969457	valid_0's ndcg@4: 0.972875	valid_0's ndcg@5: 0.974135
Early stopping,

LGBMRanker(metric='ndcg', num_boost_round=1000, objective='lambdarank')

In [6]:
gc.collect()
with open('model/model', 'wb') as f:
    pickle.dump(gbm, f)

In [7]:
with open('model/model', 'rb') as f:
    model = pickle.load(f)

In [8]:
test = pd.read_csv("data/dummy/test.csv", index_col=False)
test.drop('date_time', axis=1, inplace=True)

In [9]:
predictions = model.predict(test)

In [10]:
len(predictions)

4959183

In [11]:
submission = test[["srch_id", "prop_id"]]
submission['pred'] = predictions

<ipython-input-11-f49d9f7448f2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission['pred'] = predictions


In [12]:
del test
gc.collect()

104

In [13]:
submission = submission.sort_values(['srch_id', 'pred'], ascending=[True, False])

In [14]:
submission.head(n=20)

,srch_id,prop_id,pred
23,1,99484,0.673341
9,1,54937,0.524117
12,1,61934,0.383604
5,1,28181,0.308979
4,1,24194,0.213612
8,1,50162,0.169544
18,1,82231,0.132254
24,1,123675,0.103128
16,1,74045,0.089854
13,1,63894,0.084450


In [15]:
submission[['srch_id', 'prop_id']].to_csv('submission.csv', index=False)